<h1 style="text-align: center; font-size: 50px;"> Stock Analysis with Pandas </h1>

In this notebook, we run a series of database operations using standard Pandas, running on CPU. These values will be logged into MLFlow and used as refecence to compare with the GPU accelerated version using CUDF. 

The data we'll be working with is a subset of the [USA 514 Stocks Prices NASDAQ NYSE dataset](https://www.kaggle.com/datasets/olegshpagin/usa-stocks-prices-ohlcv) from Kaggle. This was segmented in differently sized samples, with 5M, 10M, 15M and 20M data entries, and should be set up as an asset (Dataset) called USA_Stocks on the AI Studio project. 

# Notebook Overview
- Imports
- Configurations
- Verify Assets
- Perform Analysis with Standard Pandas
- Log Results to MLflow Experiment

# Imports

In [1]:
# =============================
# Standard Library Imports
# =============================
import os
import time               # For runtime measurement (wall clock)
import logging            # For application-level logging
import warnings           # To manage and filter Python warnings
from pathlib import Path  # For object-oriented filesystem paths

# =============================
# Third-Party Library Imports
# =============================
import pandas as pd       # Data manipulation and analysis
import mlflow             # Experiment tracking and model logging

# Configurations

In [2]:
# ------------------------ Suppress Verbose Logs ------------------------
warnings.filterwarnings("ignore")

In [3]:
# Create logger
logger = logging.getLogger("data_analysis_logger")
logger.setLevel(logging.INFO)

formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s", 
                              datefmt="%Y-%m-%d %H:%M:%S")  

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.propagate = False

In [4]:
# Directory containing the USA stock parquet datasets
DATASET_DIR = Path("/home/jovyan/datafabric/USA_Stocks/")

# Sample sizes (in millions of rows) to evaluate during the analysis
SAMPLE_SIZES_TO_TEST = [5, 10]

# Rolling window size (in days) used for time-series statistical operations
ROLLING_WINDOW_SIZE = 7

# Name of the MLflow experiment for tracking performance and metrics
MLFLOW_EXPERIMENT_NAME = "USA Stock Analysis with Pandas"

In [5]:
logger.info('Notebook execution started.')

2025-04-08 19:50:03 - INFO - Notebook execution started.


# Verify Assets

In [6]:
# Define required dataset filenames
dataset_filenames = [
    "usa_stocks_5m.parquet",
    "usa_stocks_10m.parquet",
    "usa_stocks_15m.parquet",
    "usa_stocks_20m.parquet",
]

# Construct full dataset file paths using pathlib
dataset_paths = [DATASET_DIR / filename for filename in dataset_filenames]

# Check if all dataset files exist
all_files_exist = all(path.exists() for path in dataset_paths)

# Output the dataset configuration status
if all_files_exist:
    logger.info("Dataset is properly configured")
else:
    logger.info("Dataset is not properly configured. Please, create and download the assets on your project on AI Studio")

2025-04-08 19:50:03 - INFO - Dataset is properly configured


# Perform Analysis with Standard Pandas    

In the next cells, we will define functions to run different operations in datasets:
  * A function to describe the dataset
  * A function to aggregate results grouped by "ticker" (the identifier of each stock)
  * A function to aggregate by ticker, year and week
  * A function to retrieve a rolling window with a given number of days for each ticker

For each of these functions, the result will be displayed, and the necessary time to run will be logged into MLFlow. These functions will then be applied to the given set of samples in sample_sizes (e.g. [5, 10]). Bigger samples (15M and 20M) might be too heavy depending on the setup of your computer, so we recommend to configure the desired sample sizes according to the available resources.

In [7]:
def describe_dataframe(df):
    """
    Compute basic descriptive statistics for the input DataFrame.

    Parameters:
        df (pd.DataFrame): Input DataFrame.

    Returns:
        tuple: (elapsed_time_in_seconds, descriptive_statistics)
    """
    start_time = time.time()
    descriptive_stats = df.describe()
    elapsed_time = time.time() - start_time
    return elapsed_time, descriptive_stats


def aggregate_by_ticker(df):
    """
    Perform simple aggregation grouped by ticker.

    Aggregates:
        - Minimum datetime
        - Maximum datetime
        - Count of records

    Parameters:
        df (pd.DataFrame): Input DataFrame.

    Returns:
        tuple: (elapsed_time_in_seconds, aggregated_dataframe)
    """
    start_time = time.time()
    aggregation_result = df.groupby("ticker").agg({
        "datetime": ["min", "max", "count"]
    })
    elapsed_time = time.time() - start_time
    return elapsed_time, aggregation_result


def aggregate_by_ticker_week(df):
    """
    Perform composite aggregation grouped by ticker, year, and week.

    Aggregates:
        - Minimum closing price
        - Maximum closing price

    Parameters:
        df (pd.DataFrame): Input DataFrame.

    Returns:
        tuple: (elapsed_time_in_seconds, aggregated_dataframe)
    """
    start_time = time.time()
    df[["year", "week", "day"]] = df["datetime"].dt.isocalendar()
    aggregation_result = df.groupby(["ticker", "year", "week"]).agg({
        "close": ["min", "max"]
    })
    elapsed_time = time.time() - start_time
    return elapsed_time, aggregation_result


def compute_rolling_mean(df, window_days):
    """
    Calculate rolling window mean for each ticker over a given number of days.

    Parameters:
        df (pd.DataFrame): Input DataFrame.
        window_days (int): Number of days for the rolling window.

    Returns:
        tuple: (elapsed_time_in_seconds, result_dataframe)
    """
    start_time = time.time()
    result = (
        df.set_index("datetime")
          .sort_index()
          .groupby("ticker")
          .rolling(f"{window_days}D")
          .mean()
          .reset_index()
    )
    elapsed_time = time.time() - start_time
    return elapsed_time, result

# Log Results to MLflow Experiment

In [8]:
# Set the MLflow experiment to track runs
mlflow.set_experiment(experiment_name=MLFLOW_EXPERIMENT_NAME)

# Loop through each dataset sample size and run analysis
for sample_size in SAMPLE_SIZES_TO_TEST:
    run_name = f"Standard Analysis - {sample_size}M"
    
    with mlflow.start_run(run_name=run_name):
        # Log configuration parameters
        mlflow.log_param("Computing", "cpu")
        mlflow.log_param("Dataset size in millions of rows", sample_size)
        
        # Load dataset corresponding to the current sample size
        dataset_path = f"/home/jovyan/datafabric/USA_Stocks/usa_stocks_{sample_size}m.parquet"
        df = pd.read_parquet(dataset_path)

        print(f"\n--- Running Analysis for {sample_size}M Rows ---")
        
        # Description
        description_time, _ = describe_dataframe(df)
        mlflow.log_metric("Description_time_seconds", description_time)
        print(f"Description Time      : {description_time:.4f} seconds")
        
        # Simple Aggregation
        simple_agg_time, _ = aggregate_by_ticker(df)
        mlflow.log_metric("Simple_aggregation_time_seconds", simple_agg_time)
        print(f"Simple Aggregation    : {simple_agg_time:.4f} seconds")
        
        # Composite Aggregation
        composite_agg_time, _ = aggregate_by_ticker_week(df)
        mlflow.log_metric("Composite_aggregation_time_seconds", composite_agg_time)
        print(f"Composite Aggregation : {composite_agg_time:.4f} seconds")
        
        # Rolling Window
        rolling_time, _ = compute_rolling_mean(df, ROLLING_WINDOW_SIZE)
        mlflow.log_metric(f"Rolling_window_{ROLLING_WINDOW_SIZE}D_time_seconds", rolling_time)
        print(f"Rolling Window ({ROLLING_WINDOW_SIZE}D) : {rolling_time:.4f} seconds")


--- Running Analysis for 5M Rows ---
Description Time      : 0.5489 seconds
Simple Aggregation    : 0.2012 seconds
Composite Aggregation : 0.4415 seconds
Rolling Window (7D) : 3.3943 seconds

--- Running Analysis for 10M Rows ---
Description Time      : 1.2999 seconds
Simple Aggregation    : 0.3681 seconds
Composite Aggregation : 0.8355 seconds
Rolling Window (7D) : 7.9388 seconds


In [9]:
logger.info('Notebook execution completed.')

2025-04-08 19:50:21 - INFO - Notebook execution completed.


Built with ❤️ using Z by HP AI Studio.